In [144]:
import pandas as pd
dir_path =r"C:/Users/User/Desktop/finalP/final_result/robustness_test_1.5/datasets/"
df=pd.read_csv(dir_path+'(raw)Quarterly_data_1.5.csv')

In [145]:
df.head()

,quarter,future_return,volume,inventory,amount,BASIS,RSI,short_balance_ratio,spread,pli,index_trading_volume,index_turnover_rate,CCt
0,1,NaN,16.330141,13.675352,11.568298,25.292400,43.813538,0.000081,1.791637,0.360000,10.345883,0.445200,1.0
1,2,-0.021297,16.749515,14.487159,11.975821,6.743270,49.711198,0.000116,1.419721,0.603175,10.690867,0.521429,0.0
2,3,0.151496,16.503935,14.540578,11.892310,35.787000,52.878266,0.000079,1.481516,0.508197,11.336654,0.748689,0.0
3,4,-0.035664,16.282721,14.541778,11.629259,9.604879,47.325634,0.000265,1.198595,0.551724,10.978013,0.507931,0.0
4,5,-0.020328,16.138946,14.680744,11.466374,4.527033,45.282475,0.001061,0.949640,0.491803,10.672704,0.375902,0.0


In [146]:
#normalize data
cols_to_norm = ['future_return','volume','amount','BASIS','index_turnover_rate','short_balance_ratio','pli','spread','RSI','index_trading_volume','inventory']
df[cols_to_norm] = df[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
df.head()

,quarter,future_return,volume,inventory,amount,BASIS,RSI,short_balance_ratio,spread,pli,index_trading_volume,index_turnover_rate,CCt
0,1,NaN,0.561270,0.000000,0.493418,0.931309,0.324385,0.000008,1.000000,0.000000,0.086958,0.124093,1.0
1,2,0.470264,0.645924,0.297046,0.570356,0.809897,0.529036,0.000208,0.765367,0.910362,0.210244,0.169145,0.0
2,3,0.778756,0.596352,0.316592,0.554589,1.000000,0.638935,0.000000,0.804352,0.554797,0.441028,0.303457,0.0
3,4,0.444614,0.551698,0.317031,0.504927,0.828627,0.446256,0.001061,0.625864,0.717749,0.312861,0.161168,0.0
4,5,0.471993,0.522676,0.367880,0.474176,0.795391,0.375358,0.005590,0.468804,0.493426,0.203753,0.083137,0.0


In [147]:
#use SVM
from sklearn import svm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report


In [148]:
#drop NaN
df = df.dropna()
#drop week column
df = df.drop(['quarter'], axis=1)

In [149]:
from imblearn.over_sampling import SMOTE
X = df.iloc[:, :-1]  # Features (all columns except the last one)
y = df.iloc[:, -1]   # Labels (last column)
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)
balanced_df = pd.concat([X_resampled, y_resampled], axis=1)

In [150]:
balanced_df

,future_return,volume,inventory,amount,BASIS,RSI,short_balance_ratio,spread,pli,index_trading_volume,index_turnover_rate,CCt
0,0.470264,0.645924,0.297046,0.570356,0.809897,0.529036,0.000208,0.765367,0.910362,0.210244,0.169145,0.0
1,0.778756,0.596352,0.316592,0.554589,1.000000,0.638935,0.000000,0.804352,0.554797,0.441028,0.303457,0.0
2,0.444614,0.551698,0.317031,0.504927,0.828627,0.446256,0.001061,0.625864,0.717749,0.312861,0.161168,0.0
3,0.471993,0.522676,0.367880,0.474176,0.795391,0.375358,0.005590,0.468804,0.493426,0.203753,0.083137,0.0
4,0.370213,0.571028,0.372259,0.503403,0.780359,0.381434,0.010618,0.144450,0.207341,0.124871,0.033278,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
69,0.517462,0.453245,0.831522,0.454156,0.693553,0.317375,0.119793,0.478021,0.510915,0.632933,0.207600,1.0
70,0.401862,0.271999,0.551253,0.242053,0.768279,0.505806,0.146359,0.436921,0.211975,0.302364,0.051899,1.0
71,0.404061,0.481519,0.372398,0.432642,0.765349,0.308350,0.020421,0.151912,0.359216,0.202568,0.060704,1.0
72,0.474155,0.561487,0.807092,0.520576,0.704141,0.371636,0.090231,0.511720,0.445466,0.480692,0.162761,1.0


In [151]:
X = balanced_df.iloc[:, :-1]  # Features (all columns except the last one)
y = balanced_df.iloc[:, -1]   # Labels (last column)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [152]:
#fit SVM model
clf = svm.SVC(kernel='linear', C=0.1, random_state=42).fit(X_train, y_train)
y_pred = clf.predict(X_test)


In [153]:
#accuracy
print("Accuracy:",accuracy_score(y_test, y_pred))
#confusion matrix
print("Confusion Matrix:\n",confusion_matrix(y_test, y_pred))
#classification report
print("Classification Report:\n",classification_report(y_test, y_pred))

Accuracy: 0.5333333333333333
Confusion Matrix:
 [[1 7]
 [0 7]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.12      0.22         8
         1.0       0.50      1.00      0.67         7

    accuracy                           0.53        15
   macro avg       0.75      0.56      0.44        15
weighted avg       0.77      0.53      0.43        15



In [154]:
#print prediction result and comapre with y_test
print("Prediction:\n",y_pred)
print("Actual:\n",np.array(y_test))


Prediction:
 [1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
Actual:
 [1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1.]


In [155]:
#robustness
robust_500 = pd.read_csv(dir_path+'500_data_quarterly_1.5.csv')
#drop NaN
robust_500 = robust_500.dropna()
#drop week column
robust_500 = robust_500.drop(['q'], axis=1)
robust_500.head()

,volume,closingPrice,amount,BASIS,inventory,short_balance_ratio,RSI,pli,camount,turnoverRatio,spread,CCt+1
1,15.902227,-0.304442,11.690145,-310.929531,13.961356,0.003756,41.185941,0.500000,11.668560,2.949844,1.203598,0.0
2,13.366053,0.035226,9.112198,-210.375082,13.835633,0.003454,52.035060,0.590164,11.622334,2.629508,0.635253,0.0
3,13.761439,-0.220594,9.282537,-152.828983,14.223868,0.004423,37.525167,0.542373,11.006309,1.778475,0.649669,0.0
4,13.910468,0.022603,9.459649,-100.803443,14.525353,0.004796,48.791461,0.508197,11.043679,1.550164,0.723395,0.0
5,13.702239,0.064010,9.320195,-76.364219,14.556600,0.007313,40.929792,0.625000,11.038096,1.487969,0.631040,0.0


In [156]:
cols_to_norm = ['closingPrice','volume','amount','BASIS','turnoverRatio','short_balance_ratio','pli','spread','RSI','camount','inventory']
robust_500[cols_to_norm] = robust_500[cols_to_norm].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
X_robust = robust_500.iloc[:, :-1]  # Features (all columns except the last one)
y_robust = robust_500.iloc[:, -1]   # Labels (last column)
smote = SMOTE(random_state=42, k_neighbors=4)
X_resampled_robust, y_resampled_robust = smote.fit_resample(X_robust, y_robust)

#use svm to do robustness test
y_pred_robust = clf.predict(X_resampled_robust)
print("Accuracy:",accuracy_score(y_resampled_robust, y_pred_robust))
print("Confusion Matrix:\n",confusion_matrix(y_resampled_robust, y_pred_robust))
print("Classification Report:\n",classification_report(y_resampled_robust, y_pred_robust))

Accuracy: 0.5416666666666666
Confusion Matrix:
 [[ 2 22]
 [ 0 24]]
Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      0.08      0.15        24
         1.0       0.52      1.00      0.69        24

    accuracy                           0.54        48
   macro avg       0.76      0.54      0.42        48
weighted avg       0.76      0.54      0.42        48

